In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io
import seaborn as sns

from dosipy.utils.viz import save_fig

# head

In [ ]:
frequencies = {5: 10}
distances = [2.5, 5, 10, 50, 150]
cp = 7
# simulation
adult = pd.read_csv(os.path.join('data', 'adult_head.csv'))
adult = adult[adult['col_points'] == cp]
child = pd.read_csv(os.path.join('data', 'child_head.csv'))
child = child[child['col_points'] == cp]
# stochastic analysis
mean_adult_mat = scipy.io.loadmat(os.path.join('data', f'adult_M_SC{cp}'))
var_adult_mat = scipy.io.loadmat(os.path.join('data', f'adult_V_SC{cp}'))
mean_child_mat = scipy.io.loadmat(os.path.join('data', f'child_M_SC{cp}'))
var_child_mat = scipy.io.loadmat(os.path.join('data', f'child_V_SC{cp}'))

In [ ]:
f = frequencies[5]
mu_adult = mean_adult_mat[f'M_SC{cp}'][5]
mu_child = mean_child_mat[f'M_SC{cp}'][5]
var_adult = var_adult_mat[f'V_SC{cp}'][5]
var_child = var_child_mat[f'V_SC{cp}'][5]
f_cond_adult = adult['frequency [GHz]'] == f
y_ref_adult = adult[f_cond_adult]['sPDn_pln [W/m2]'].unique()
f_cond_child = child['frequency [GHz]'] == f
y_ref_child = child[f_cond_child]['sPDn_pln [W/m2]'].unique()
assert (y_ref_adult == y_ref_child).all()
y_ref = y_ref_child.copy()
child_adult_perc_diff = (mu_child - mu_adult) / y_ref * 100
adult_planar_perc_diff = (mu_adult - y_ref) / y_ref * 100
child_planar_perc_diff = (mu_child - y_ref) / y_ref * 100

In [ ]:
sns.set_context('talk', font_scale=1.1, rc={'lines.linewidth': 3,
                                            'lines.markersize': 12})
c = sns.color_palette(palette='colorblind', n_colors=3)

fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot()
ax.hlines(y=1, xmin=distances[0], xmax=distances[-1], color='k', lw=2)
ax.text(distances[0], 0.55, 'margin of\nnegligible effect', size=12)
ax.plot(distances, y_ref, 'o-', c=c[0], mew=3, mfc='w',
        label='planar')
ax.errorbar(distances, mu_adult, yerr=3*np.sqrt(var_adult),
            ls='--', marker='o', c=c[1], mew=3, mfc='w',
            label='adult, 99% CI')
ax.errorbar(distances, mu_child, yerr=3*np.sqrt(var_child),
            ls=':', marker='d', c=c[2], mew=3, mfc='w',
        label='child, 99% CI')
ax.set(xlabel='distance [mm]',
       ylabel='incident PD [W/m$^2$]',
       xscale='log',
       xticks=distances,
       xticklabels=distances,
       yticks=(0, 4, 8),
       yticklabels=(0, 4, 8),
       ylim=(-1.25, 9.25))
ax.legend()

sns.despine()
fig.tight_layout()

fname = 'presentation_head_IPD'
save_fig(fig, os.path.join('figures', fname))

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot()
ax.plot(distances, child_adult_perc_diff, 'o-', c=c[0], mew=3, mfc='w',
        label='child - adult')
ax.plot(distances, adult_planar_perc_diff, 'x--', c=c[1], mew=3, mfc='w',
        label='adult - planar')
ax.plot(distances, child_planar_perc_diff, 'd:', c=c[2], mew=3, mfc='w',
        label='child - planar')
ax.set(xlabel='distance [mm]',
       ylabel='incident PD % diff.',
       xscale='log',
       xticks=distances,
       xticklabels=distances,
       yticks=(0, 6, 12),
       yticklabels=(0, 6, 12),
       ylim=(-2, 14))
ax.legend()

sns.despine()
fig.tight_layout()

fname = 'presentation_head_diff'
save_fig(fig, os.path.join('figures', fname))

# curvature effect

In [ ]:
frequency = 10
distances = [2.5, 5, 10, 50, 150]
radii = [1.8, 18]
curvature = pd.read_csv(os.path.join('data', 'curvature.csv'))

In [ ]:
f_cond = curvature['frequency [GHz]'] == frequency
extracted_curvature = curvature[f_cond]
r_cond_small = extracted_curvature['radius [cm]'] == min(radii)
r_cond_big = extracted_curvature['radius [cm]'] == max(radii)
y_small = extracted_curvature[r_cond_small]['sPDn_sph [W/m2]'].to_numpy()
y_big = extracted_curvature[r_cond_big]['sPDn_sph [W/m2]'].to_numpy()
y_ref_1 = extracted_curvature[r_cond_small]['sPDn_pln [W/m2]'].to_numpy()
y_ref_2 = extracted_curvature[r_cond_big]['sPDn_pln [W/m2]'].to_numpy()
assert (y_ref_1 == y_ref_2).all()
y_ref = y_ref_1.copy()
small_big_perc_diff = (y_small - y_big) / y_ref * 100
big_planar_perc_diff = (y_big - y_ref) / y_ref * 100
small_planar_perc_diff = (y_small - y_ref) / y_ref * 100
d_nf = 3e8 / (f * 1e9) / (2 * np.pi) * 1000

In [ ]:
sns.set_context('talk', font_scale=1.1, rc={'lines.linewidth': 3,
                                             'lines.markersize': 12})
c = sns.color_palette(palette='colorblind', n_colors=3)

fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot()
ax.plot(distances, y_ref, 'o-', c=c[0], mew=3, mfc='w',
        label='planar')
ax.plot(distances, y_big, 'x--', c=c[1], mew=3, mfc='w',
        label='$r_{max}$ = 18 cm')
ax.plot(distances, y_small, 'd:', c=c[2], mew=3, mfc='w',
        label='$r_{min}$ = 1.8 cm')
ax.set(xlabel='distance [mm]',
       ylabel='incident PD [W/m$^2$]',
       xscale='log',
       xticks=distances,
       xticklabels=distances,
       yticks=(0, 5, 10),
       yticklabels=(0, 5, 10),
       ylim=(-1, 11))
ax.legend()

sns.despine()
fig.tight_layout()

In [ ]:
fname = 'presentation_curvature_IPD'
save_fig(fig, os.path.join('figures', fname))

fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot()
ax.vlines(x=d_nf, ymin=-12, ymax=24, color='k', lw=2)
ax.text(distances[0], -10, 'radiative\nnear field', rotation=90, size=12)
ax.plot(distances, small_big_perc_diff, 'o-', c=c[0], mew=3, mfc='w',
        label='$r_{min}$ - $r_{max}$')
ax.plot(distances, big_planar_perc_diff, 'x--', c=c[1], mew=3, mfc='w',
        label='$r_{max}$ - planar')
ax.plot(distances, small_planar_perc_diff, 'd:', c=c[2], mew=3, mfc='w',
        label='$r_{min}$ - planar')
ax.set(xlabel='distance [mm]',
       ylabel='incident PD % diff.',
       xscale='log',
       xticks=distances,
       xticklabels=distances,
       yticks=(-12, 0, 12, 24),
       yticklabels=(-12, 0, 12, 24),
       ylim=(-14.4, 26.4))
ax.legend()

sns.despine()
fig.tight_layout()

fname = 'presentation_curvature_diff'
save_fig(fig, os.path.join('figures', fname))